In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import collections
import datetime
import time


from matplotlib import font_manager, rc
from matplotlib import style
style.use('ggplot')
%matplotlib inline

font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

pd.set_option('display.max_rows', 220)
pd.set_option('display.max_columns', 10)

In [2]:
data_busan=pd.read_csv('./data/소상공인시장진흥공단_상가(상권)정보_부산_20200630.csv', encoding="cp949",error_bad_lines=False)

In [3]:
want_data_busan=data_busan[(data_busan['상권업종대분류코드']=='D')&(data_busan['상권업종중분류명']=='종합소매점')]
df1_1=want_data_busan[want_data_busan['상권업종소분류명']=='수퍼마켓']
df2_1=want_data_busan[want_data_busan['상권업종소분류명']=='편의점']
df3_1=want_data_busan[want_data_busan['상권업종소분류명']=='종합소매']
df4_1=want_data_busan[want_data_busan['상권업종소분류명']=='할인점']
want_data_busan=pd.concat([df1_1, df2_1, df3_1, df4_1], axis=0)
want_data_busan=want_data_busan.reset_index()
want_data_busan['구분코드']=''

In [4]:
for i in range(len(want_data_busan['상호명'])):
    if (pd.isnull(want_data_busan['지점명'][i])):
        want_data_busan['지점명'][i]='없음'

In [5]:
for i in range(len(want_data_busan['상호명'])):
    if (('GS25' in want_data_busan['상호명'][i]) or ('CU' in want_data_busan['상호명'][i]) 
        or ('세븐일레븐' in want_data_busan['상호명'][i]) or ('미니스톱' in want_data_busan['상호명'][i]) 
        or ('이마트24' in want_data_busan['상호명'][i]) or (str(24) in want_data_busan['지점명'][i])):
        want_data_busan['구분코드'][i]='편의점'

In [6]:
want_data_busan

,index,상가업소번호,상호명,지점명,상권업종대분류코드,...,층정보,호정보,경도,위도,구분코드
0,91,22186288,롯데캐슬리버,없음,D,...,1.0,NaN,128.974521,35.127894,
1,185,22112565,삼일슈퍼,없음,D,...,1.0,NaN,128.958714,35.103845,
2,243,24793955,삼한할인슈퍼,없음,D,...,1.0,NaN,129.018781,35.235175,
3,483,20045800,희망상회앞,없음,D,...,NaN,NaN,129.082161,35.184688,
4,496,20043942,알짜마트,없음,D,...,NaN,NaN,129.036446,35.124388,
...,...,...,...,...,...,...,...,...,...,...,...
6987,152254,16435590,이마트24,없음,D,...,NaN,NaN,129.107067,35.138051,편의점
6988,152357,16431720,이마트,24구서SK뷰점,D,...,NaN,NaN,129.086777,35.244354,편의점
6989,152559,16468637,이마트,블루웨,D,...,NaN,703.0,129.061385,35.196284,
6990,153962,16546543,이마트,24일광해수욕장점,D,...,1.0,NaN,129.232648,35.261957,편의점


In [7]:
for i in range(len(want_data_busan['상호명'])):
    if (('INCUPS' in want_data_busan['상호명'][i]) or ('CAFFEPASCUCCI' in want_data_busan['상호명'][i]) or
       ('보드게임카페CUBE' in want_data_busan['상호명'][i]) or ('하프타임' in want_data_busan['상호명'][i]) or
       ('에프엠CUSTOM' in want_data_busan['상호명'][i]) or ('GCUT' in want_data_busan['상호명'][i])):
        want_data_busan['구분코드'][i]='슈퍼마켓'

In [8]:
# for i in range(len(want_data_busan['상호명'])):
#     if ('이마트' in want_data_busan['상호명'][i]) and (str(24) not in want_data_busan['상호명'][i]):
#         if (str(24) not in want_data_busan['지점명'][i]):

In [9]:
e_list=[107, 850, 1771, 2252, 5161, 5272, 5967, 6439, 6666, 6676, 6694, 6777, 6782, 6805, 6830, 6831, 6874, 6878, 6918, 6928, 6989]
for i in e_list:
    want_data_busan.loc[i]['구분코드']='대형마트'

In [10]:
for i in range(len(want_data_busan['상호명'])):
    if (('롯데마트' in want_data_busan['상호명'][i]) or ('홈플러스' in want_data_busan['상호명'][i]) 
        or ('하나로마트' in want_data_busan['상호명'][i]) or ('메가마트' in want_data_busan['상호명'][i]) 
        or ('코스트코' in want_data_busan['상호명'][i]) or ('탑마트' in want_data_busan['상호명'][i])):
        want_data_busan['구분코드'][i]='대형마트'

In [11]:
for i in range(len(want_data_busan['상호명'])):
    if (want_data_busan['구분코드'][i]==0):
        want_data_busan['구분코드'][i]='슈퍼마켓'

In [12]:
seper_index=[]
seper_code=[]
for i in range(len(want_data_busan['구분코드'])):
    seper_index.append(want_data_busan.loc[i]['index'])
    seper_code.append(want_data_busan.loc[i]['구분코드'])

In [13]:
want_data_busan['구분코드']

0          
1          
2          
3          
4          
       ... 
6987    편의점
6988    편의점
6989       
6990    편의점
6991    편의점
Name: 구분코드, Length: 6992, dtype: object

In [14]:
data_busan['구분코드']=''

In [15]:
#loc[i].[구분코드]는 적용되지 않지만, [구분코드].loc[i]는 먹는 이유가 뭘까
j=0
for i in seper_index:
    data_busan['구분코드'].loc[i]=seper_code[j]
    j+=1

In [16]:
data_busan.to_csv('./data/save/상가정보_부산_코드구분.csv',encoding='euc-kr')

In [17]:
data_busan['구분코드'].unique()

array(['', '편의점', '대형마트', '슈퍼마켓'], dtype=object)

In [18]:
s=pd.Series([1, 3, 5, np.nan, 6, 8])

In [19]:
help(print)

Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.

